# Limpieza de modelo de riesgos

In [16]:
import pandas as pd
import dask
dask.config.set({'dataframe.query-planning': True})
import re
from dataprep.clean import clean_text
from nltk.corpus import stopwords

In [17]:
df = pd.read_csv("../data/raw/loan.csv")
df.head()

C:\Users\JUAN\AppData\Local\Temp\ipykernel_11228\1708148095.py:1: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/loan.csv")


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.shape

(887379, 74)

In [19]:
for column in df.columns:    
    unique_values = len(df[column].unique())
    
    if unique_values > df.shape[0] * 0.75:
        print(f"Columna {column} con muchos valores únicos:", unique_values)
        df.drop(column, axis=1, inplace= True)
        continue
    
    if unique_values == 1:
        print(f"Columna {column} con un solo valor único")
        df.drop(column, axis=1, inplace= True)
        continue

Columna id con muchos valores únicos: 887379
Columna member_id con muchos valores únicos: 887379
Columna url con muchos valores únicos: 887379
Columna policy_code con un solo valor único


In [20]:
threshold = 0.75
columns_to_drop = df.columns[df.isnull().mean() > threshold]
print(f"Columns to drop: {columns_to_drop}")

df_without_nulls = df.drop(columns_to_drop, axis=1)

Columns to drop: Index(['desc', 'mths_since_last_record', 'mths_since_last_major_derog',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl',
       'inq_last_12m'],
      dtype='object')


In [21]:
df_without_nulls.shape

(887379, 50)

In [22]:
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

def convert_to_datetime(date_str):
    try:
        date_str = date_str.strip()
        
        month_str, year_str = date_str.split('-')
        month_num = month_map.get(month_str.strip(), None)
        if month_num:
            return pd.to_datetime(f"{year_str}-{month_num}-01")
        else:
            return pd.NaT 
    except Exception:
        return pd.NaT

In [24]:
df_cleaned = df_without_nulls.copy()
df_cleaned["desc"] = df["desc"]
for col in df_cleaned.select_dtypes(include=["object"]).columns:
    sample_value = df_cleaned[col].dropna().iloc[0]
    if re.match(r"\w{3}-[\d]{4}$", str(sample_value)):
        df_cleaned[col] = df_cleaned[col].map(convert_to_datetime)
    elif col in ("term", "emp_length"):
        df_cleaned[col] = df_cleaned[col].str.extract(r"(\d+)")
    else:
        df_cleaned[col] = clean_text(df_cleaned, col)[col]


In [26]:
df_cleaned.to_csv("../data/interim/loan_cleaned.csv", index=False)